In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-26 22:12:29,132 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [3]:
pushgateway_client = infrastructure_container.pushgateway_client()
prometheus_hpc_admin_client = infrastructure_container.prometheus_hpc_admin_client()

pbs_pro_resources_used_pusher = infrastructure_container.pbs_pro_resources_used_pusher()
gpu_stats_pusher = infrastructure_container.gpu_stats_pusher()

In [4]:
job_id = '561566'

In [ ]:
import asyncio


async def push():
    while True:
        try:
            await pbs_pro_resources_used_pusher.push(job_id)
            await gpu_stats_pusher.push()
            await asyncio.sleep(60)

        except Exception as e:
            print(e)
            break


await push()

In [ ]:
# PUSHGATEWAY_JOB = 'gpu_stat'

# await pushgateway_client.delete_job(PUSHGATEWAY_JOB)

# matchers = [f'pbs_job_cpu_percent{{job_id="{job_id}"}}', f'pbs_job_mem_bytes{{job_id="{job_id}"}}']
# await prometheus_hpc_admin_client.delete_series(matchers)

In [ ]:
await pushgateway_client.delete_job('gpu_stat')

matchers = ['gpu_stats_util_percent', 'gpu_stats_mem_bytes']
await prometheus_hpc_admin_client.delete_series(matchers)